In [63]:
import tensorflow as tf
import snap7
from snap7 import util
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
#client = snap7.client.Client()
#client.connect('169.254.113.134',0,1)
import pathlib
os.chdir("D:\Camera Roll")

#images_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
#train_images, train_labels = next(images_generator.flow_from_directory("D:\Camera Roll"))


#dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
#data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
#data_dir = pathlib.Path(data_dir)
batch_size = 6
img_height = 256
img_width = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  
    directory='D:\Camera Roll\TrainingData',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height, img_width))



val_ds = tf.keras.utils.image_dataset_from_directory(
    directory='D:\Camera Roll\ValidationData',
    labels='inferred',
    label_mode='categorical',
    batch_size=batch_size,
    image_size=(img_height, img_width))

class_names = train_ds.class_names
print(class_names)
    
    

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))



data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    
num_classes = len(class_names)
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

#loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#model = keras.applications.Xception(weights=None, input_shape=(256, 256, 3), classes=2)
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


#model.summary()

#model.fit(train_ds, epochs=10, validation_data=val_ds)
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Found 60 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
['Bad', 'Good']
0.37467834 1.0
Epoch 1/10
10/10 [==============================] - 2s 151ms/step - loss: 1.4826 - accuracy: 0.6667 - val_loss: 0.5816 - val_accuracy: 0.7500
Epoch 2/10
10/10 [==============================] - 1s 123ms/step - loss: 0.5881 - accuracy: 0.7333 - val_loss: 0.5632 - val_accuracy: 0.7500
Epoch 3/10
10/10 [==============================] - 1s 115ms/step - loss: 0.5824 - accuracy: 0.7333 - val_loss: 0.5616 - val_accuracy: 0.7500
Epoch 4/10
10/10 [==============================] - 1s 118ms/step - loss: 0.5822 - accuracy: 0.7333 - val_loss: 0.5648 - val_accuracy: 0.7500
Epoch 5/10
10/10 [==============================] - 1s 118ms/step - loss: 0.5804 - accuracy: 0.7333 - val_loss: 0.5642 - val_accuracy: 0.7500
Epoch 6/10
10/10 [==============================] - 1s 126ms/step - loss: 0.5793 - accuracy: 0.7333 - val_loss: 0.5615 - val_accuracy: 0.7500
Epoch 7/10
10/10 [=====================

In [64]:
img = tf.keras.utils.load_img(
    'D:\G_Img_20.jpg', target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to Bad with a 100.00 percent confidence.


In [130]:
import snap7
from snap7 import util
client = snap7.client.Client()
client.connect('192.168.50.10',0,1)
#print(client.get_connected())

client.db_read(1, 0, 2)
db = client.db_read(1, 0, 2)


t = snap7.util.get_bool(db, 0,0)
print(t)
p= snap7.util.set_bool(db,0,0,1)
t = snap7.util.get_bool(db, 0,0)
print(t)


False
True
